<a href="https://colab.research.google.com/github/kalyani234/TextGeneration_AIAgent/blob/CrewAI/ArticleCreation_AI_Agent_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Project Overview: AI Article Generation using CrewAI and Groq LLM

This project demonstrates how to create an AI-powered multi-agent writing system using **CrewAI** and **Groq’s llama-3.1-8b-instant model**.
The goal is to automatically generate and refine well-structured articles based on any topic you input — similar to how a content team works in real life.

It uses:

* CrewAI: to simulate a team of AI agents (Writer and Editor).

* Groq API: to access a fast, instruction-tuned large language model.

* Google Colab: for interactive workflow execution and file downloads.

🔍 Workflow Summary

* Writer Agent → generates a draft article based on the topic.

* Editor Agent → reviews, refines, and polishes the article.

* Output → final article with improved readability, logical flow, and style.

* Result → article saved and downloaded as final_article.txt.

This step installs the required Python packages:
- `crewai`: a framework for creating multi-agent AI workflows.
- `crewai-tools`: additional utilities to extend CrewAI's capabilities.

The `--no-cache-dir` flag prevents cached installations for a clean setup, and `-q` suppresses verbose logs for clarity.


In [ ]:
# Install required packages
!pip install crewai crewai-tools -q --no-cache-dir


This section imports essential libraries:
- `crewai` for agent and task management.
- `google.colab.userdata` to securely retrieve stored API keys.
- `os` for environment variable handling.
- `time` for adding short pauses.

It then attempts to load the Groq API key from Colab’s `userdata` and sets it as both `GROQ_API_KEY` and `OPENAI_API_KEY`
because CrewAI uses the OpenAI-style interface for LLMs.

Here we configure the Large Language Model (LLM) using Groq’s endpoint.

Parameters:
- `model="llama-3.1-8b-instant"` → Groq’s llama-3.1-8b instruction-tuned model.
- `api_key` → your Groq API key.
- `base_url` → Groq’s OpenAI-compatible API endpoint.
- `temperature=0.7` → adds slight creativity to responses.

Then, a quick test query ("Say 'Connection successful'") ensures the API connection works correctly.


In [ ]:
from crewai import Agent, Task, Crew, LLM
from google.colab import userdata
import os
import time

# Set up Groq API
try:
    groq_api_key = userdata.get('GROQ_API_KEY')
    os.environ["GROQ_API_KEY"] = groq_api_key
    os.environ["OPENAI_API_KEY"] = groq_api_key  # CrewAI also expects this
    print(" Groq API key loaded successfully")
except Exception as e:
    print(f" Error loading API key: {e}")

# --- Configure LLM for CrewAI with Groq ---
try:
    llm = LLM(
        model="llama-3.1-8b-instant",        # Fast Groq model
        api_key=os.environ["GROQ_API_KEY"],  # Pass your key
        base_url="https://api.groq.com/openai/v1",  # Required for Groq
        temperature=0.7
    )

    print("🚀 CrewAI LLM configured successfully with Groq!")

except Exception as e:
    print(f"❌ Error initializing LLM: {e}")
    raise e


Before generating content, the script asks the user to input a topic for the article.

Example input: "**The History of Renewable Energy**".

If input fails, a fallback topic "Default Topic" is used.  
A short delay is added beforehand to ensure the Colab input cell displays correctly.


In [ ]:

# Ensure input prompt is visible
print("\nPlease wait a moment for the input prompt to appear...")
time.sleep(1)  # Delay to stabilize the interface



In [ ]:
# Prompt user for topic with a clear message
try:
    topic = input("Please enter the topic for the article (e.g., The History of Renewable Energy): ").strip()
    print(f"Selected topic: {topic}")  # Confirm the input
except Exception as e:
    print(f"Error capturing topic input: {e}")
    topic = "Default Topic"  # Fallback if input fails

The **Writer Agent** acts as the article creator.

Attributes:
- **Role**: “Content Writer”
- **Goal**: Create engaging, structured articles on the selected topic with image placeholders.
- **Backstory**: Mimics a professional digital content writer who knows how to simplify complex information.
- **LLM**: Uses the configured Groq model.
- **Verbose**: Enabled to show detailed logs during execution.

The **Editor Agent** ensures content quality.

Attributes:
- **Role**: “Chief Editor”
- **Goal**: Maintain clarity, flow, factual accuracy, and tone consistency.
- **Backstory**: Acts as a seasoned editor with strong editorial judgment.
- **Verbose**: Enabled to log its review process.

This agent will review and improve the Writer’s output while preserving image placeholders.


In [ ]:
# Define Agents with the configured LLM
writer = Agent(
    role="Content Writer",
    goal=f"Create engaging, well-structured articles about {topic} with relevant image placeholders",
    backstory="""You are an experienced content writer with expertise in creating compelling articles for digital platforms.
    You excel at breaking down complex topics into engaging, easy-to-understand content and know exactly where to place
    visual elements to enhance reader engagement.""",
    llm=llm,
    verbose=True
)

editor = Agent(
    role="Chief Editor",
    goal="Ensure the highest quality standards in all published content",
    backstory="""You are a meticulous editor with years of experience in digital publishing. You have a keen eye for detail,
    strong sense of narrative flow, and deep understanding of what makes content engaging and trustworthy for readers.""",
    llm=llm,
    verbose=True
)

The Writing Task instructs the Writer Agent to:
- Generate a 400-word article.
- Include a title, intro, main sections, and a conclusion.
- Insert 2–3 image placeholders using the format:
  [Image: descriptive caption].
- Use an engaging, accessible tone.

Expected Output: a plain text article draft.

The Editing Task instructs the Editor Agent to:
- Review and refine the Writer’s draft.
- Improve clarity, logical flow, tone, and readability.
- Maintain factual accuracy.
- Preserve all image placeholders as-is.

Expected Output: a polished plain text article.

The `context=[writing_task]` parameter ensures the editor receives the writer’s output as input.


In [ ]:

# Define the Writing Task
writing_task = Task(
    description=f"Write a compelling article on {topic}. Create a compelling title and introduction. Structure the content with clear headings and subheadings. Include 3-4 substantial paragraphs covering different aspects: key facts, historical context (if relevant), and current perspectives. Add 2-3 strategic image placeholders using the format: [Image: descriptive caption about what should be visualized]. Ensure the tone is engaging and accessible to a general audience. End with a concise conclusion that summarizes key points. Limit the response to 400 words.",
    expected_output="A plain text draft article as a string",
    agent=writer
)

# Define the Editing Task
editing_task = Task(
    description=f"Review the draft article on {topic} received from the Writer. Follow this checklist: ✓ Improve clarity and readability ✓ Ensure logical flow between sections ✓ Verify factual accuracy and balance ✓ Enhance engagement and reader appeal ✓ Check for consistent tone and style ✓ Preserve ALL image placeholders exactly as: [Image: descriptive caption] ✓ Eliminate any grammatical errors or awkward phrasing ✓ Ensure the content is well-structured and easy to follow. Do not remove or modify the image placeholders. Keep them exactly in their original format. Return the final version as plain text.",
    expected_output="A polished plain text article as a string",
    agent=editor,
    context=[writing_task]  # Pass the Writer's output as context
)

Here, both agents and their tasks are grouped into a `Crew` object.

- **Agents**: Writer and Editor.
- **Tasks**: Writing and Editing.
- **Process**: Sequential (the Editor waits for the Writer’s draft).

Then, `crew.kickoff()` starts the full workflow.  
The script prints progress logs and captures both draft and final versions.

After the workflow completes:
1. The final edited article is written to a text file named `final_article.txt`.
2. The file is automatically offered for download using Colab’s file utility.

Error handling ensures informative messages appear if any part of this process fails.



In [ ]:
# Assemble the Crew
crew = Crew(
    agents=[writer, editor],
    tasks=[writing_task, editing_task],
    verbose=True,
    process="sequential"
)

# Kick off the crew with progress and output checks
try:
    print("Starting crew execution... (This may take a few minutes)")
    result = crew.kickoff()
    print(f"Crew execution completed. Result: {result}")

    # Extract and verify draft and final content
    draft_content = result.get('tasks', {}).get(writing_task.id, {}).get('output', 'No draft content found')
    final_content = result.get('tasks', {}).get(editing_task.id, {}).get('output', 'No final content found')

    print(f"Draft content: {draft_content}")
    print(f"Final content: {final_content}")

    # Save final content to file for download
    try:
        with open('final_article.txt', 'w') as f:
            f.write(final_content)
        print("'final_article.txt' created successfully.")
    except Exception as e:
        print(f"Error creating 'final_article.txt': {e}")

    # Download the final article
    try:
        from google.colab import files
        if os.path.exists('final_article.txt'):
            files.download('final_article.txt')
            print("'final_article.txt' downloaded successfully.")
        else:
            print("'final_article.txt' not found for download.")
    except Exception as e:
        print(f"Error downloading file: {e}")

except Exception as e:
    print(f"Error during execution: {e}")